In [2]:
import fitz
import numpy as np
import pandas as pd
from tqdm import tqdm

import io
from PIL import Image, ImageDraw, ImageFont, ImageColor

import math
import re

### IMPORTING BOOKS

In [3]:
vol1_path = '../input/NOUVELLE FLORE DU LIBAN ET DE LA SYRIE 1.pdf'
vol2_path = '../input/NOUVELLE FLORE DU LIBAN ET DE LA SYRIE 2.pdf'
vol3_path = '../input/NOUVELLE FLORE DU LIBAN ET DE LA SYRIE 3.pdf'

vol1_doc = fitz.open(vol1_path)
vol2_doc = fitz.open(vol2_path)
vol3_doc = fitz.open(vol3_path)

vol1_pages = [vol1_doc[i] for i in range(vol1_doc.page_count)]
vol2_pages = [vol2_doc[i] for i in range(vol2_doc.page_count)]
vol3_pages = [vol3_doc[i] for i in range(vol3_doc.page_count)]

In [4]:
vol1_char_df = pd.read_pickle("../input/char_df/vol1_df.pkl")
vol2_char_df = pd.read_pickle("../input/char_df/vol2_df.pkl")
vol3_char_df = pd.read_pickle("../input/char_df/vol3_df.pkl")

vol1_index = list(range(616, 639)) #inclusive
vol2_index = list(range(703, 725))
vol3_index = list(range(555, 583))

#### Setting Global parameters

In [5]:
TARGET_DPI = 300
mat = fitz.Matrix(TARGET_DPI/ 72, TARGET_DPI/ 72)

### Finding strict matching genera, epithet, and column numbers

In [6]:
def genus_match(row):
    word_rspace_removed = row['word']
    return row['word_num'] == 0 and \
           word_rspace_removed.isalpha() and \
           word_rspace_removed[0].isupper() and word_rspace_removed[1:].islower()
           
def epithet_match(row):
    word_rspace_removed = row['word']
    return row['word_num'] == 0 and \
           word_rspace_removed.isalpha() and \
           word_rspace_removed.islower()

In [7]:
#rightmost point of any bounding box:
def get_center_x0(vol_char_df, page_num, bias = 30):
    """WARNING: Bias = 30 large bias causes miscatagorization in page number in book"""
    df = vol_char_df[vol_char_df['page_num'] == page_num]
    
    right_bound = df['line_bbox'].apply(lambda x : x[2]).max() 
    #leftmost point of any bounding box:
    left_bound = df['line_bbox'].apply(lambda x : x[0]).min()

    return 0.5*(right_bound + left_bound) - bias


def get_col_num(coords, center_x0):
    x0, y0, x1, y1 = coords
    return int(x0 >= center_x0)

In [8]:
all_vol_data_col_num = [(vol1_char_df, vol1_index, vol1_doc),
                        (vol2_char_df, vol2_index, vol2_doc),
                        (vol3_char_df, vol3_index, vol3_doc)]

for vol_char_df ,vol_index, doc in all_vol_data_col_num: 
    #for each volume check if genus pattern / epithet pattern exists within the index part of the book
    vol_char_df['genus_index_pat_match'] = (vol_char_df['page_num'].isin(vol_index)) & (vol_char_df.apply(genus_match, axis = 1))
    vol_char_df['epithet_index_pat_match'] = (vol_char_df['page_num'].isin(vol_index)) & (vol_char_df.apply(epithet_match, axis = 1))
    
    for page_num in tqdm(vol_index):
        center_x0 = get_center_x0(vol_char_df, page_num)
        #find center based on x0 coordinate of each line
        vol_char_df['col_num'] = vol_char_df['line_bbox'].apply(lambda coords : get_col_num(coords, center_x0)) 

100%|██████████| 28/28 [00:14<00:00,  1.96it/s]


### Genus / epithet flagging 
flagging pages where number of strict genus or epithet patern matches is less than 3 per column

In [9]:
all_vol_data_flagg_strict_match = [(vol1_char_df, vol1_index, vol1_doc, "strickt_match_vol1"),
                                   (vol2_char_df, vol2_index, vol2_doc, "strickt_match_vol2"),
                                   (vol3_char_df, vol3_index, vol3_doc, "strickt_match_vol3")]

for vol_char_df, vol_index, vol_doc, output_name in all_vol_data_flagg_strict_match: 
    #for each volume 
    image_list = []
    genus_flag_list = []
    epithet_flag_list = []
    for page_num in tqdm(vol_index):
        pix_map = vol_doc.get_page_pixmap(page_num,matrix=mat)
        image = Image.open(io.BytesIO(pix_map.tobytes()))
        draw = ImageDraw.Draw(image)

        genus_db = vol_char_df[(vol_char_df['page_num'] == page_num)
                                & (vol_char_df['genus_index_pat_match'] == True)
                            ].loc[:,~vol_char_df.columns.isin(["char_num", "char", "char_origin",	"char_bbox"])
                            ].drop_duplicates()

        epithet_db = vol_char_df[(vol_char_df['page_num'] == page_num) 
                                & (vol_char_df['epithet_index_pat_match'] == True)
                                ].loc[:,~vol_char_df.columns.isin(["char_num", "char", "char_origin",	"char_bbox"])
                                ].drop_duplicates()

        #genus pattern match flag should check with half page and not entire page:
        for col in range(2):
            num_genus_col = genus_db[genus_db["col_num"] == col].shape[0]
            num_epithet_col = epithet_db[epithet_db["col_num"] == col].shape[0]
            if num_genus_col <= 2:
                genus_flag_list.append((num_genus_col, page_num - vol_index[0] + 1, col))
            if num_epithet_col <= 2:
                epithet_flag_list.append((num_epithet_col, page_num - vol_index[0] + 1, col))

        for coord in genus_db['word_bbox']:
            x0, y0, x1, y1 = [f*TARGET_DPI/ 72 for f in coord]
            draw.rectangle((x0, y0, x1, y1), fill=None, outline=ImageColor.getrgb("#FF7F50"), width=5)

        for coord in epithet_db['word_bbox']:
            x0, y0, x1, y1 = [f*TARGET_DPI/ 72 for f in coord]
            draw.rectangle((x0, y0, x1, y1), fill=None, outline=ImageColor.getrgb("#003399"), width=5)

        image_list.append(image)

    image_list[0].save('../output/local/'+output_name+'.pdf' ,save_all=True, append_images=image_list[1:])    
    
    num_flag_pages = len(set([g[1] for g in genus_flag_list] + [e[1] for e in epithet_flag_list]))
    if num_flag_pages > 0: 
        print("***FLAGS***")
        print(f" number of pages to check: {num_flag_pages}")
        if genus_flag_list:
            print("  genera")
            [print(f"\t number of genera: {g_flag[0]}, page number: {g_flag[1]}, column number: {g_flag[2]}") for g_flag in genus_flag_list]
        if epithet_flag_list:
            print("  epithets")
            [print(f"\t number of epithets: {e_flag[0]}, page number: {e_flag[1]}, column number: {e_flag[2]}") for e_flag in epithet_flag_list]

100%|██████████| 23/23 [00:04<00:00,  5.64it/s]


***FLAGS***
 number of pages to check: 3
  genera
	 number of genera: 1, page number: 2, column number: 0
	 number of genera: 0, page number: 20, column number: 1
	 number of genera: 1, page number: 23, column number: 0
  epithets
	 number of epithets: 2, page number: 23, column number: 1


100%|██████████| 22/22 [00:03<00:00,  5.61it/s]


***FLAGS***
 number of pages to check: 2
  genera
	 number of genera: 2, page number: 4, column number: 0
	 number of genera: 1, page number: 4, column number: 1
	 number of genera: 0, page number: 5, column number: 0


100%|██████████| 28/28 [00:04<00:00,  5.91it/s]


***FLAGS***
 number of pages to check: 7
  genera
	 number of genera: 1, page number: 2, column number: 1
	 number of genera: 1, page number: 6, column number: 0
	 number of genera: 1, page number: 21, column number: 0
	 number of genera: 1, page number: 22, column number: 0
	 number of genera: 2, page number: 24, column number: 1
	 number of genera: 0, page number: 26, column number: 1
	 number of genera: 2, page number: 28, column number: 0


Based on flags need to make sure: 
- first find epithet coord match 
- then find genus coord match s.t. word is not in epithet coord match

### match based on coordinates

In [10]:
def is_coord_match(x, x_ref_left, x_ref_right, margin):
    return (x_ref_left - margin <= x[0] and x[0] <= x_ref_left + margin) or (x_ref_right - margin <= x[0] and x[0] <= x_ref_right + margin)

#### epithets

In [11]:
all_vol_data_coord_match = [(vol1_char_df, vol1_index),
                            (vol2_char_df, vol2_index),
                            (vol3_char_df, vol3_index)]

for vol_char_df, vol_index in all_vol_data_coord_match: 
    vol_char_df["epithet_coord_match"] = vol_char_df["word_bbox"].apply(lambda x : False)
    for page_num in tqdm(vol_index):
        margin = 1.25 * vol_char_df[(vol_char_df["epithet_index_pat_match"] == True)]["char_bbox"].apply(lambda x : x[2] - x[0]).mean()
        epithet_char_df = vol_char_df[(vol_char_df["page_num"] == page_num) & (vol_char_df["epithet_index_pat_match"] == True)]
        epithet_df = epithet_char_df.loc[:,~vol_char_df.columns.isin(["char_num", "char", "char_origin", "char_bbox"])].drop_duplicates()
        page_epithet_2dic = [{}, {}]
        
        for i in range(epithet_df.shape[0]):
            e_index = str(page_num) + "_" + str(i)
            p0 = epithet_df['word_bbox'].iloc[i]
            x_ref = p0[0]
            col = epithet_df['col_num'].iloc[i]

            ref_neighbors_df = epithet_df[(epithet_df["page_num"] == page_num) & 
                                          (epithet_df["word_bbox"].apply(lambda x : x_ref - margin <= x[0] and x[0] <= x_ref + margin))]
            
            num_neighbors = ref_neighbors_df.shape[0]
            mean_neighbors = ref_neighbors_df["word_bbox"].apply(lambda x : x[0]).mean()
            page_epithet_2dic[col][e_index] = (num_neighbors, mean_neighbors)
        
        mean_left_epithet = max(page_epithet_2dic[0].values(), default = [-1, -1])[1]
        mean_right_epithet = max(page_epithet_2dic[1].values(), default = [-1, -1])[1]

        if mean_left_epithet == -1 or mean_right_epithet == -1:
            mean_valid_col = max(mean_left_epithet, mean_right_epithet)
            vol_char_df.loc[(vol_char_df["page_num"] == page_num) , "epithet_coord_match"] = vol_char_df[(vol_char_df["page_num"] == page_num)]["pruned_word_bbox"].apply(lambda x : is_coord_match(x, mean_valid_col, mean_valid_col, margin))
        elif mean_left_epithet == -1 and mean_right_epithet == -1:
            vol_char_df.loc[(vol_char_df["page_num"] == page_num) , "epithet_coord_match"] = vol_char_df[(vol_char_df["page_num"] == page_num)]["pruned_word_bbox"].apply(lambda x : False)
        else: 
            vol_char_df.loc[(vol_char_df["page_num"] == page_num) , "epithet_coord_match"] = vol_char_df[(vol_char_df["page_num"] == page_num)]["pruned_word_bbox"].apply(lambda x : is_coord_match(x, mean_left_epithet, mean_right_epithet, margin))

100%|██████████| 28/28 [00:01<00:00, 24.65it/s]


In [12]:
all_vol_data_epithet_coord_match_test = [(vol1_char_df, vol1_index, vol1_doc, "epithet_coord_match_pruned_vol1"),
                                         (vol2_char_df, vol2_index, vol2_doc, "epithet_coord_match_pruned_vol2"),
                                         (vol3_char_df, vol3_index, vol3_doc, "epithet_coord_match_pruned_vol3")]

for vol_char_df, vol_index, doc, output_name in all_vol_data_epithet_coord_match_test: 
    #for each volume 
    image_list = []
    
    for page_num in tqdm(vol_index):
        pix_map = doc.get_page_pixmap(page_num,matrix=mat)
        image = Image.open(io.BytesIO(pix_map.tobytes()))
        draw = ImageDraw.Draw(image)
        
        epithet_coord_db = vol_char_df[(vol_char_df['page_num'] == page_num) & 
                                     (vol_char_df['epithet_coord_match'] == True)
                            ].loc[:,~vol_char_df.columns.isin(["char_num", "char", "char_origin",	"char_bbox"])
                            ].drop_duplicates()

        epithet_db = vol_char_df[(vol_char_df['page_num'] == page_num) 
                                & (vol_char_df['epithet_index_pat_match'] == True)
                                ].loc[:,~vol_char_df.columns.isin(["char_num", "char", "char_origin",	"char_bbox"])
                                ].drop_duplicates()

        #epithet Coord is orange-pinkish, 5
        for coord in epithet_coord_db["pruned_word_bbox"] :
            x0, y0, x1, y1 = [f*TARGET_DPI/ 72 for f in coord]
            draw.rectangle((x0, y0, x1, y1), fill=None, outline=ImageColor.getrgb("#FF7F50"), width=5)

        #epithet is blue, 3
        for coord in epithet_db['word_bbox'] :
            x0, y0, x1, y1 = [f*TARGET_DPI/ 72 for f in coord]
            draw.rectangle((x0, y0, x1, y1), fill=None, outline=ImageColor.getrgb("#003399"), width=3)
        image_list.append(image)

    #save pages of the volume
    image_list[0].save('../output/local/'+output_name+'.pdf' ,save_all=True, append_images=image_list[1:])

100%|██████████| 28/28 [00:04<00:00,  5.95it/s]


In [13]:
# Reminder:
# all_vol_data_coord_match = [(vol1_char_df, vol1_index),
#                             (vol2_char_df, vol2_index),
#                             (vol3_char_df, vol3_index)]
# DOES NOT CHECK IF COORD IS SAME AS EPITHET UNTIL NEXT SECTION!

for vol_char_df, vol_index in all_vol_data_coord_match: 
    #genus and not epithet
    vol_char_df["genus_coord_match"] = vol_char_df["word_bbox"].apply(lambda x : False)
    for page_num in tqdm(vol_index):
        margin = 1.25 * vol_char_df[(vol_char_df["genus_index_pat_match"] == True)]["char_bbox"].apply(lambda x : x[2] - x[0]).mean()
        genus_char_df = vol_char_df[(vol_char_df["page_num"] == page_num) &
                                    (vol_char_df["genus_index_pat_match"] == True)]
        genus_df = genus_char_df.loc[:,~vol_char_df.columns.isin(["char_num", "char", "char_origin", "char_bbox"])].drop_duplicates()
        page_genus_2dic = [{}, {}]
        
        epithet_left_coord_mean = vol_char_df[(vol_char_df["epithet_coord_match"] == True) &
                                              (vol_char_df["page_num"] == page_num) &
                                              (vol_char_df["col_num"] == 0)
                                             ]['pruned_word_bbox'].apply(lambda x : x[0]).mean()
        epithet_right_coord_mean = vol_char_df[(vol_char_df["epithet_coord_match"] == True) &
                                               (vol_char_df["page_num"] == page_num) &
                                               (vol_char_df["col_num"] == 1)
                                             ]['pruned_word_bbox'].apply(lambda x : x[0]).mean()
        epithet_coord_mean_list = [epithet_left_coord_mean, epithet_right_coord_mean]

        for i in range(genus_df.shape[0]):
            g_index = str(page_num) + "_" + str(i)
            p0 = genus_df['word_bbox'].iloc[i]
            x_ref = p0[0]
            col = genus_df['col_num'].iloc[i]

            ref_neighbors_df = genus_df[(genus_df["page_num"] == page_num) & 
                                        (genus_df["word_bbox"].apply(lambda x : x_ref - margin <= x[0] and x[0] <= x_ref + margin))]

            num_neighbors = ref_neighbors_df.shape[0]
            mean_neighbors = ref_neighbors_df["word_bbox"].apply(lambda x : x[0]).mean()
            if mean_neighbors > epithet_coord_mean_list[col]: 
                mean_neighbors = -1
            page_genus_2dic[col][g_index] = (num_neighbors, mean_neighbors)
        
        mean_left_genus = max(page_genus_2dic[0].values(), default = [-1, -1])[1]
        mean_right_genus = max(page_genus_2dic[1].values(), default = [-1, -1])[1]

        if mean_left_genus == -1 or mean_right_genus == -1:
            mean_valid_col = max(mean_left_genus, mean_right_genus)
            vol_char_df.loc[(vol_char_df["page_num"] == page_num) , "genus_coord_match"] = vol_char_df[(vol_char_df["page_num"] == page_num)]["pruned_word_bbox"].apply(lambda x : is_coord_match(x, mean_valid_col, mean_valid_col, margin))
        elif mean_left_genus == -1 and mean_right_genus == -1:
            vol_char_df.loc[(vol_char_df["page_num"] == page_num) , "genus_coord_match"] = vol_char_df[(vol_char_df["page_num"] == page_num)]["pruned_word_bbox"].apply(lambda x : False)
        else: 
            vol_char_df.loc[(vol_char_df["page_num"] == page_num) , "genus_coord_match"] = vol_char_df[(vol_char_df["page_num"] == page_num)]["pruned_word_bbox"].apply(lambda x : is_coord_match(x, mean_left_genus, mean_right_genus, margin))

100%|██████████| 28/28 [00:00<00:00, 33.49it/s]


In [14]:
all_vol_data_genus_coord_match_test = [(vol1_char_df, vol1_index, vol1_doc, "genus_coord_match_vol1"),
                                       (vol2_char_df, vol2_index, vol2_doc, "genus_coord_match_vol2"),
                                       (vol3_char_df, vol3_index, vol3_doc, "genus_coord_match_vol3")]

for vol_char_df, vol_index, doc, output_name in all_vol_data_genus_coord_match_test: 
    #for each volume 
    image_list = []

    for page_num in tqdm(vol_index):
        pix_map = doc.get_page_pixmap(page_num,matrix=mat)
        image = Image.open(io.BytesIO(pix_map.tobytes()))
        draw = ImageDraw.Draw(image)
        

        genus_coord_db = vol_char_df[(vol_char_df['page_num'] == page_num) & 
                                     (vol_char_df['genus_coord_match'] == True)
                            ].loc[:,~vol_char_df.columns.isin(["char_num", "char", "char_origin",	"char_bbox"])
                            ].drop_duplicates()

        epithet_db = vol_char_df[(vol_char_df['page_num'] == page_num) 
                                & (vol_char_df['epithet_coord_match'] == True)
                                ].loc[:,~vol_char_df.columns.isin(["char_num", "char", "char_origin",	"char_bbox"])
                                ].drop_duplicates()

        #genus Coord is orange-pinkish, 5
        for coord in genus_coord_db['word_bbox'] :
            x0, y0, x1, y1 = [f*TARGET_DPI/ 72 for f in coord]
            draw.rectangle((x0, y0, x1, y1), fill=None, outline=ImageColor.getrgb("#FF7F50"), width=5)
            
        # #epithet is red, 3
        for coord in epithet_db['word_bbox'] :
            x0, y0, x1, y1 = [f*TARGET_DPI/ 72 for f in coord]
            draw.rectangle((x0, y0, x1, y1), fill=None, outline=ImageColor.getrgb("#000099"), width=3)
        image_list.append(image)

    #save pages of the volume
    image_list[0].save('../output/local/'+output_name+'.pdf' ,save_all=True, append_images=image_list[1:])

100%|██████████| 28/28 [00:04<00:00,  5.98it/s]


#### improving the coord matches 
takes genus coming before epithet into account now

In [15]:
def potential_genus_match(row):
    word_rspace_removed = row['word']
    return row['genus_coord_match'] == True and \
           row['epithet_coord_match'] == False and \
           word_rspace_removed.find("Flore") == -1 and \
           ((word_rspace_removed.isupper() == False and \
             word_rspace_removed.isnumeric() == False) or \
            ((word_rspace_removed == 'X') or (word_rspace_removed =='×')))
           # removing this for-    hg now ... and row['genus_mean_coord'] < row['epithet_mean_coord'] #important to check this only when epithet_coord_match is false?

def potential_epithet_match(row):
    word_rspace_removed = row['word']
    return row['epithet_coord_match'] == True and \
           ((word_rspace_removed.isupper() == False and \
             word_rspace_removed.isnumeric() == False) or \
            (word_rspace_removed == 'X') or (word_rspace_removed =='×'))

In [16]:
vol1_char_df['potential_genus_match'] = vol1_char_df.apply(potential_genus_match, axis = 1)
vol1_char_df['potential_epithet_match'] = vol1_char_df.apply(potential_epithet_match, axis = 1)

vol2_char_df['potential_genus_match'] = vol2_char_df.apply(potential_genus_match, axis = 1)
vol2_char_df['potential_epithet_match'] = vol2_char_df.apply(potential_epithet_match, axis = 1)

vol3_char_df['potential_genus_match'] = vol3_char_df.apply(potential_genus_match, axis = 1)
vol3_char_df['potential_epithet_match'] = vol3_char_df.apply(potential_epithet_match, axis = 1)

In [17]:
all_vol_data_GE_potential_match_test = [(vol1_char_df, vol1_index, vol1_doc, "GE_potential_match_vol1"),
                                        (vol2_char_df, vol2_index, vol2_doc, "GE_potential_match_vol2"),
                                        (vol3_char_df, vol3_index, vol3_doc, "GE_potential_match_vol3")]

for vol_char_df, vol_index, doc, output_name in all_vol_data_GE_potential_match_test: 
    #for each volume 
    image_list = []

    for page_num in tqdm(vol_index):
        pix_map = doc.get_page_pixmap(page_num,matrix=mat)
        image = Image.open(io.BytesIO(pix_map.tobytes()))
        draw = ImageDraw.Draw(image)
        
        genus_db = vol_char_df[(vol_char_df['page_num'] == page_num) & 
                                     (vol_char_df['potential_genus_match'] == True)
                            ].loc[:,~vol_char_df.columns.isin(["char_num", "char", "char_origin",	"char_bbox"])
                            ].drop_duplicates()

        epithet_db = vol_char_df[(vol_char_df['page_num'] == page_num) 
                                & (vol_char_df['potential_epithet_match'] == True)
                                ].loc[:,~vol_char_df.columns.isin(["char_num", "char", "char_origin",	"char_bbox"])
                                ].drop_duplicates()

        #genus Coord is orange-pinkish, 5
        for coord in genus_db['word_bbox'] :
            x0, y0, x1, y1 = [f*TARGET_DPI/ 72 for f in coord]
            draw.rectangle((x0, y0, x1, y1), fill=None, outline=ImageColor.getrgb("#FF7F50"), width=5)
            
        # #epithet is red, 3
        for coord in epithet_db['word_bbox'] :
            x0, y0, x1, y1 = [f*TARGET_DPI/ 72 for f in coord]
            draw.rectangle((x0, y0, x1, y1), fill=None, outline=ImageColor.getrgb("#000099"), width=3)
        image_list.append(image)

    #save pages of the volume
    image_list[0].save('../output/local/'+output_name+'.pdf' ,save_all=True, append_images=image_list[1:])

100%|██████████| 28/28 [00:04<00:00,  6.07it/s]


In [ ]:
# must take or of flag properties:
def flags_decomposer(flags):
    """Make font flags human readable."""
    l = []
    if flags & 2 ** 0:
        l.append("superscript")
    if flags & 2 ** 1:
        l.append("italic")
    if flags & 2 ** 2:
        l.append("serifed")
    else:
        l.append("sans")
    if flags & 2 ** 3:
        l.append("monospaced")
    else:
        l.append("proportional")
    if flags & 2 ** 4:
        l.append("bold")
    return ", ".join(l)

span_flags = []
result = 0
for b in span_flags:
    result = span_flags | result

### PIPELINES! 

In [18]:
import sklearn

In [21]:
from sklearn.base import BaseEstimator, TransformerMixin

In [22]:
vol1_char_df.columns

Index(['vol_num', 'page_num', 'block_num', 'block_num_absolute', 'block_bbox',
       'line_num', 'line_wmode', 'line_dir', 'line_bbox', 'span_num',
       'span_size', 'span_flags', 'span_font', 'span_color', 'span_ascender',
       'span_descender', 'span_origin', 'span_bbox', 'word_num', 'word',
       'word_bbox', 'pruned_word', 'pruned_word_bbox', 'char_num', 'char',
       'char_origin', 'char_bbox', 'genus_index_pat_match',
       'epithet_index_pat_match', 'col_num', 'epithet_coord_match',
       'genus_coord_match', 'potential_genus_match',
       'potential_epithet_match'],
      dtype='object')

In [25]:
for col_name in vol1_char_df.columns:
    len(vol1_char_df[col_name].unique()) == 1 

### PIPELINES END

### SOME HARDCODING PARTS:

Erianthus

In [56]:
vol1_char_df[vol1_char_df['word'].str.contains('d,IlLIlU')]

,vol_num,page_num,block_num,block_num_absolute,block_bbox,line_num,line_wmode,line_dir,line_bbox,span_num,...,char,char_origin,char_bbox,genus_index_pat_match,epithet_index_pat_match,col_num,epithet_coord_match,genus_coord_match,potential_genus_match,potential_epithet_match
1731817,1,624,75,120,"(230.63999938964844, 352.70001220703125, 265.1...",0,0,"(1.0, 0.0)","(230.63999938964844, 352.70001220703125, 265.1...",1,...,J,"(239.9900665283203, 357.20001220703125)","(239.9900665283203, 352.70001220703125, 242.33...",False,False,1,True,False,False,True
1731818,1,624,75,120,"(230.63999938964844, 352.70001220703125, 265.1...",0,0,"(1.0, 0.0)","(230.63999938964844, 352.70001220703125, 265.1...",1,...,.,"(242.32186889648438, 357.20001220703125)","(242.32186889648438, 352.70001220703125, 243.8...",False,False,1,True,False,False,True
1731819,1,624,75,120,"(230.63999938964844, 352.70001220703125, 265.1...",0,0,"(1.0, 0.0)","(230.63999938964844, 352.70001220703125, 265.1...",1,...,d,"(243.81723022460938, 357.20001220703125)","(243.81723022460938, 352.70001220703125, 246.8...",False,False,1,True,False,False,True
1731820,1,624,75,120,"(230.63999938964844, 352.70001220703125, 265.1...",0,0,"(1.0, 0.0)","(230.63999938964844, 352.70001220703125, 265.1...",1,...,",","(246.8169708251953, 357.20001220703125)","(246.8169708251953, 352.70001220703125, 248.32...",False,False,1,True,False,False,True
1731821,1,624,75,120,"(230.63999938964844, 352.70001220703125, 265.1...",0,0,"(1.0, 0.0)","(230.63999938964844, 352.70001220703125, 265.1...",1,...,I,"(248.3123321533203, 357.20001220703125)","(248.3123321533203, 352.70001220703125, 250.31...",False,False,1,True,False,False,True
1731822,1,624,75,120,"(230.63999938964844, 352.70001220703125, 265.1...",0,0,"(1.0, 0.0)","(230.63999938964844, 352.70001220703125, 265.1...",1,...,l,"(250.30714416503906, 357.20001220703125)","(250.30714416503906, 352.70001220703125, 251.9...",False,False,1,True,False,False,True
1731823,1,624,75,120,"(230.63999938964844, 352.70001220703125, 265.1...",0,0,"(1.0, 0.0)","(230.63999938964844, 352.70001220703125, 265.1...",1,...,L,"(251.9709930419922, 357.20001220703125)","(251.9709930419922, 352.70001220703125, 255.64...",False,False,1,True,False,False,True
1731824,1,624,75,120,"(230.63999938964844, 352.70001220703125, 265.1...",0,0,"(1.0, 0.0)","(230.63999938964844, 352.70001220703125, 265.1...",1,...,I,"(255.638671875, 357.20001220703125)","(255.638671875, 352.70001220703125, 257.642547...",False,False,1,True,False,False,True
1731825,1,624,75,120,"(230.63999938964844, 352.70001220703125, 265.1...",0,0,"(1.0, 0.0)","(230.63999938964844, 352.70001220703125, 265.1...",1,...,l,"(257.63348388671875, 357.20001220703125)","(257.63348388671875, 352.70001220703125, 259.3...",False,False,1,True,False,False,True
1731826,1,624,75,120,"(230.63999938964844, 352.70001220703125, 265.1...",0,0,"(1.0, 0.0)","(230.63999938964844, 352.70001220703125, 265.1...",1,...,U,"(259.2973327636719, 357.20001220703125)","(259.2973327636719, 352.70001220703125, 263.64...",False,False,1,True,False,False,True


based on this image output in volumen 1:
 ![Erianthus](Erianthus.png)



In [57]:
weird_old_char_vol1 = vol1_char_df.loc[1731810:1731830]
weird_old_char_vol1['word']

1731810          Bieb.
1731811          Bieb.
1731812          Bieb.
1731813           J_JI
1731814           J_JI
1731815           J_JI
1731816           J_JI
1731817    J.d,IlLIlU.
1731818    J.d,IlLIlU.
1731819    J.d,IlLIlU.
1731820    J.d,IlLIlU.
1731821    J.d,IlLIlU.
1731822    J.d,IlLIlU.
1731823    J.d,IlLIlU.
1731824    J.d,IlLIlU.
1731825    J.d,IlLIlU.
1731826    J.d,IlLIlU.
1731827    J.d,IlLIlU.
1731828         hostii
1731829         hostii
1731830         hostii
Name: word, dtype: object

In [76]:
vol_num_Erianthus1, page_num_Erianthus1, block_num_Erianthus1, line_num_Erianthus1, span_num_Erianthus1, word_num_Erianthus1 = vol1_char_df[vol1_char_df['word'].str.contains('J_JI')][['vol_num', 'page_num', 'block_num', 'line_num', 'span_num', 'word_num']].iloc[0]

In [77]:
vol1_char_df.columns

Index(['vol_num', 'page_num', 'block_num', 'block_num_absolute', 'block_bbox',
       'line_num', 'line_wmode', 'line_dir', 'line_bbox', 'span_num',
       'span_size', 'span_flags', 'span_font', 'span_color', 'span_ascender',
       'span_descender', 'span_origin', 'span_bbox', 'word_num', 'word',
       'word_bbox', 'pruned_word', 'pruned_word_bbox', 'char_num', 'char',
       'char_origin', 'char_bbox', 'genus_index_pat_match',
       'epithet_index_pat_match', 'col_num', 'epithet_coord_match',
       'genus_coord_match', 'potential_genus_match',
       'potential_epithet_match'],
      dtype='object')

In [78]:
vol_num_Erianthu2, page_num_Erianthus2, block_num_Erianthus2, line_num_Erianthus2, span_num_Erianthus2, word_num_Erianthus2 = vol1_char_df[vol1_char_df['word'].str.contains('J.d,IlLIlU.')][['vol_num', 'page_num', 'block_num', 'line_num', 'span_num','word_num']].iloc[0]

In [84]:
target_index = vol1_char_df[((vol1_char_df['vol_num'] == vol_num_Erianthus1) | (vol1_char_df['vol_num'] == vol_num_Erianthu2)) &
                            ((vol1_char_df['page_num'] == page_num_Erianthus1) | (vol1_char_df['page_num'] == page_num_Erianthus2)) &
                            ((vol1_char_df['block_num'] == block_num_Erianthus1) | (vol1_char_df['block_num'] == block_num_Erianthus2)) & 
                            ((vol1_char_df['line_num'] == line_num_Erianthus1) | (vol1_char_df['line_num'] == line_num_Erianthus2)) &  
                            ((vol1_char_df['span_num'] == span_num_Erianthus1) | (vol1_char_df['span_num'] == span_num_Erianthus2)) & 
                            ((vol1_char_df['word_num'] == word_num_Erianthus1) | (vol1_char_df['word_num'] == word_num_Erianthus2)) 
                            ].index

In [86]:
start_index = 1731813
end_index = 1731827
weird_old_char_vol1 = vol1_char_df.loc[target_index]
weird_old_char_vol1['word']

1731813           J_JI
1731814           J_JI
1731815           J_JI
1731816           J_JI
1731817    J.d,IlLIlU.
1731818    J.d,IlLIlU.
1731819    J.d,IlLIlU.
1731820    J.d,IlLIlU.
1731821    J.d,IlLIlU.
1731822    J.d,IlLIlU.
1731823    J.d,IlLIlU.
1731824    J.d,IlLIlU.
1731825    J.d,IlLIlU.
1731826    J.d,IlLIlU.
1731827    J.d,IlLIlU.
Name: word, dtype: object

make comment about this part is hard coded thingi

In [87]:
weird_old_char_vol1['word_num']

1731813    0
1731814    0
1731815    0
1731816    0
1731817    1
1731818    1
1731819    1
1731820    1
1731821    1
1731822    1
1731823    1
1731824    1
1731825    1
1731826    1
1731827    1
Name: word_num, dtype: int64

In [89]:
#manually fixing the OCR error for J_JI J.d,IlLIlU. hostii Griseb.
vol1_char_df.loc[target_index, 'word'] = 'Erianthus'
vol1_char_df.loc[target_index, 'word_num'] = 0
vol1_char_df.loc[target_index, 'pruned_word'] = 'Erianthus'
temp_word_x0 = vol1_char_df.loc[target_index, 'word_bbox'].apply(lambda x : x[0]).min()
temp_word_y0 = vol1_char_df.loc[target_index, 'word_bbox'].apply(lambda x : x[1]).min()
temp_word_x1 = vol1_char_df.loc[target_index, 'word_bbox'].apply(lambda x : x[2]).max()
temp_word_y1 = vol1_char_df.loc[target_index, 'word_bbox'].apply(lambda x : x[3]).max()
vol1_char_df.loc[target_index, 'word_bbox'] =vol1_char_df.loc[target_index, 'word_bbox'].apply(lambda x : (temp_word_x0, temp_word_y0, temp_word_x1, temp_word_y1))

vol1_char_df.loc[target_index, 'potential_epithet_match'] = False
vol1_char_df.loc[target_index, 'potential_genus_match'] = True

### Infra species

In [90]:
# Reminder:
# all_vol_data_coord_match = [(vol1_char_df, vol1_index),
#                             (vol2_char_df, vol2_index),
#                             (vol3_char_df, vol3_index)]
for vol_char_df, vol_index in all_vol_data_coord_match: 
    vol_char_df["infra_coord_match"] = vol_char_df["word_bbox"].apply(lambda x : False)
    for page_num in tqdm(vol_index):

        margin = 1.25 * vol_char_df[(vol_char_df["potential_epithet_match"] == True) | (vol_char_df["potential_genus_match"] == True)]["char_bbox"].apply(lambda x : x[2] - x[0]).mean()
        
        mean_left_epithet = vol_char_df[(vol_char_df["page_num"] == page_num) & (vol_char_df["col_num"] == 0) & (vol_char_df["potential_epithet_match"] == True)]["word_bbox"].apply(lambda x : x[0]).mean()
        mean_left_genus = vol_char_df[(vol_char_df["page_num"] == page_num) & (vol_char_df["col_num"] == 0) & (vol_char_df["potential_genus_match"] == True)]["word_bbox"].apply(lambda x : x[0]).mean()
        if math.isnan(mean_left_genus):
            mean_left_genus_all = vol_char_df[(vol_char_df["col_num"] == 0) & (vol_char_df["potential_genus_match"] == True)]["word_bbox"].apply(lambda x : x[0]).mean()
            mean_left_epithet_all = vol_char_df[(vol_char_df["col_num"] == 0) & (vol_char_df["potential_epithet_match"] == True)]["word_bbox"].apply(lambda x : x[0]).mean()
            mean_left_tab = mean_left_epithet_all - mean_left_genus_all
        else: 
            mean_left_tab = mean_left_epithet - mean_left_genus
        
        mean_right_epithet = vol_char_df[(vol_char_df["page_num"] == page_num) & (vol_char_df["col_num"] == 1) & (vol_char_df["potential_epithet_match"] == True)]["word_bbox"].apply(lambda x : x[0]).mean()
        mean_right_genus = vol_char_df[(vol_char_df["page_num"] == page_num) & (vol_char_df["col_num"] == 1) & (vol_char_df["potential_genus_match"] == True)]["word_bbox"].apply(lambda x : x[0]).mean()
        if math.isnan(mean_right_genus):
            mean_right_genus_all = vol_char_df[(vol_char_df["col_num"] == 1) & (vol_char_df["potential_genus_match"] == True)]["word_bbox"].apply(lambda x : x[0]).mean()
            mean_right_epithet_all = vol_char_df[(vol_char_df["col_num"] == 1) & (vol_char_df["potential_epithet_match"] == True)]["word_bbox"].apply(lambda x : x[0]).mean()
            mean_right_tab = mean_right_epithet_all - mean_right_genus_all
        else: 
            mean_right_tab = mean_right_epithet - mean_right_genus


        vol_char_df.loc[(vol_char_df["page_num"] == page_num) & (vol_char_df["word_num"] == 0)  , "infra_coord_match"] = vol_char_df[(vol_char_df["page_num"] == page_num) & (vol_char_df["word_num"] == 0)]["word_bbox"].apply(lambda x : is_coord_match(x, mean_left_epithet + mean_left_tab, mean_right_epithet + mean_right_tab, margin))

100%|██████████| 28/28 [00:01<00:00, 20.26it/s]


In [91]:
# Takes longer but makes more sense generally. We will skip it here
# def potential_author_match_infra_coord(row):
#     word = row['word']
#     pruned_word = row['pruned_word']
#     lower_word = word.lower()
#     latin_connectives = r"^\s?et[\s|.]?$|^\s?in[\s|.]?$|^\s?non[\s|.]?$|^\s?&[\s|.]?$|^\s?er[\s|.]?$|^\s?nec[\s|.]?$|^\s?mult[\s|.]?$|^\s?ex[\s|.]?$|^\s?fil[\s|.]?$"
#     infra_symbols = r"^var[\s|.|\b]?$|^subsp[\s|.|\b]?$|^ssp[\s|.|\b]?$|^spp[\s|.|\b]?$|^x[\s|.|\b]?$|^×[\s|.|\b]?$"
#     is_latin_connectives = re.search(latin_connectives, word) != None
#     is_infra_symbol = re.search(infra_symbols, lower_word) != None
#     if pruned_word:
#         is_upper_first = pruned_word[0].isupper()
#     else:
#         is_upper_first = False
#     return (not is_infra_symbol) and (is_upper_first or is_latin_connectives)

def potential_author_match_infra_coord(word):
    lower_word = word.lower()
    latin_connectives = r"^\s?et[\s|.]?$|^\s?in[\s|.]?$|^\s?non[\s|.]?$|^\s?&[\s|.]?$|^\s?er[\s|.]?$|^\s?nec[\s|.]?$|^\s?mult[\s|.]?$|^\s?ex[\s|.]?$|^\s?fil[\s|.]?$"
    infra_symbols = r"^var[\s|.|\b]?$|^subsp[\s|.|\b]?$|^ssp[\s|.|\b]?$|^spp[\s|.|\b]?$|^x[\s|.|\b]?$|^×[\s|.|\b]?$"
    is_latin_connectives = re.search(latin_connectives, word) != None
    is_infra_symbol = re.search(infra_symbols, lower_word) != None
    return (not is_infra_symbol) and (word[0].isupper() or is_latin_connectives)

In [92]:
def has_infra_symbols(word):
    infra_symbols = r"^var[\s|.|\b]?$|^subsp[\s|.|\b]?$|^ssp[\s|.|\b]?$|^spp[\s|.|\b]?$|^x[\s|.|\b]?$|^×[\s|.|\b]?$"
    return re.search(infra_symbols, word) != None

In [93]:
# Reminder:
# all_vol_data_coord_match = [(vol1_char_df, vol1_index),
#                             (vol2_char_df, vol2_index),
#                             (vol3_char_df, vol3_index)]
for vol_char_df, _ in all_vol_data_coord_match:
    vol_char_df["potential_infra_match"] = (vol_char_df['word'].apply(has_infra_symbols)) | \
                                           ((vol_char_df["infra_coord_match"] == True) & (vol_char_df['word'].apply(potential_author_match_infra_coord) == False))

In [94]:
all_vol_dat_infra_match_test = [(vol1_char_df, vol1_index, vol1_doc, "potential_infra_match_vol1"),
                                (vol2_char_df, vol2_index, vol2_doc, "potential_infra_match_vol2"),
                                (vol3_char_df, vol3_index, vol3_doc, "potential_infra_match_vol3")][::-1]

for vol_char_df, vol_index, doc, output_name in all_vol_dat_infra_match_test: 
    #for each volume 
    image_list = []

    for page_num in tqdm(vol_index):
        pix_map = doc.get_page_pixmap(page_num,matrix=mat)
        image = Image.open(io.BytesIO(pix_map.tobytes()))
        draw = ImageDraw.Draw(image)
        

        infra_coord_db = vol_char_df[(vol_char_df['page_num'] == page_num) & 
                                     (vol_char_df['infra_coord_match'] == True)
                            ].loc[:,~vol_char_df.columns.isin(["char_num", "char", "char_origin",	"char_bbox"])
                            ].drop_duplicates()

        infra_db = vol_char_df[(vol_char_df['page_num'] == page_num) 
                                & (vol_char_df['potential_infra_match'] == True)
                                ].loc[:,~vol_char_df.columns.isin(["char_num", "char", "char_origin",	"char_bbox"])
                                ].drop_duplicates()

        with_infra_symbols = vol_char_df[(vol_char_df['page_num'] == page_num) &
                                         (vol_char_df['infra_coord_match'] == True) & 
                                         (vol_char_df['word'].apply(has_infra_symbols) == True)
                                        ].loc[:,~vol_char_df.columns.isin(["char_num", "char", "char_origin",	"char_bbox"])
                                        ].drop_duplicates()

        #genus Coord is orange-pinkish, 5
        for coord in infra_coord_db['word_bbox'] :
            x0, y0, x1, y1 = [f*TARGET_DPI/ 72 for f in coord]
            draw.rectangle((x0-5, y0-5, x1+5, y1+5), fill=None, outline=ImageColor.getrgb("#003399"), width=7)

        for coord in infra_db['word_bbox'] :
            x0, y0, x1, y1 = [f*TARGET_DPI/ 72 for f in coord]
            draw.rectangle((x0-3, y0-3, x1+3, y1+3), fill=None, outline=ImageColor.getrgb("#FF7F50"), width=5)
            
        # #epithet is red, 3
        for coord in with_infra_symbols['word_bbox'] :
            x0, y0, x1, y1 = [f*TARGET_DPI/ 72 for f in coord]
            draw.rectangle((x0, y0, x1, y1), fill=None, outline=ImageColor.getrgb("#990000"), width=3)

        image_list.append(image)

    #save pages of the volume
    image_list[0].save('../output/local/'+output_name+'.pdf' ,save_all=True, append_images=image_list[1:])

100%|██████████| 23/23 [00:20<00:00,  1.12it/s]


### page num processings

In [95]:
vol1_char_df['index_page_num'] = vol1_char_df['page_num'] - vol1_index[0] + 1
vol2_char_df['index_page_num'] = vol2_char_df['page_num'] - vol2_index[0] + 1
vol3_char_df['index_page_num'] = vol3_char_df['page_num'] - vol3_index[0] + 1

In [96]:
# all_vol_data_col_num = [(vol1_char_df, vol1_index, vol1_doc),
#                         (vol2_char_df, vol2_index, vol2_doc),
#                         (vol3_char_df, vol3_index, vol3_doc)]

for vol_char_df ,vol_index, vol_doc in all_vol_data_col_num: 
    #for each volume check if genus pattern / epithet pattern exists within the index part of the book
    for page_num in tqdm(vol_index):
        center_x0 = get_center_x0(vol_char_df, page_num, - 30)
        #find center based on x0 coordinate of each line
        vol_char_df['col_num_for_PN'] = vol_char_df['line_bbox'].apply(lambda coords : get_col_num(coords, center_x0)) 

100%|██████████| 28/28 [00:14<00:00,  1.96it/s]


In [97]:
def is_page_num(row):
    return row['pruned_word'].isnumeric()

In [98]:
# Reminder:
# all_vol_data_coord_match = [(vol1_char_df, vol1_index),
#                             (vol2_char_df, vol2_index),
#                             (vol3_char_df, vol3_index)]
for vol_char_df, vol_index in all_vol_data_coord_match: 
    vol_char_df['page_num_index_pat_match'] = (vol_char_df['page_num'].isin(vol_index)) & (vol_char_df.apply(is_page_num, axis = 1))
    vol_char_df["page_num_coord_match"] = vol_char_df["word_bbox"].apply(lambda x : False)
    for page_num in tqdm(vol_index):
        margin = 1.25 * vol_char_df[(vol_char_df["page_num_index_pat_match"] == True)]["char_bbox"].apply(lambda x : x[2] - x[0]).mean()
        page_num_char_df = vol_char_df[(vol_char_df["page_num"] == page_num) & (vol_char_df["page_num_index_pat_match"] == True)]
        page_num_df = page_num_char_df.loc[:,~vol_char_df.columns.isin(["char_num", "char", "char_origin", "char_bbox"])].drop_duplicates()
        page_page_num_2dic = [{}, {}]
        
        for i in range(page_num_df.shape[0]):
            e_index = str(page_num) + "_" + str(i)
            p0 = page_num_df['word_bbox'].iloc[i]
            x_ref = p0[2]
            col = page_num_df['col_num_for_PN'].iloc[i]

            ref_neighbors_df = page_num_df[(page_num_df["page_num"] == page_num) & 
                                           (page_num_df["word_bbox"].apply(lambda x : x_ref - margin <= x[2] and x[2] <= x_ref + margin))]
            
            num_neighbors = ref_neighbors_df.shape[0]
            mean_neighbors = ref_neighbors_df["word_bbox"].apply(lambda x : x[2]).mean()
            page_page_num_2dic[col][e_index] = (num_neighbors, mean_neighbors)
        
        mean_left_page_num = max(page_page_num_2dic[0].values(), default = [-1, -1])[1]
        mean_right_page_num = max(page_page_num_2dic[1].values(), default = [-1, -1])[1]

        if mean_left_page_num == -1 or mean_right_page_num == -1:
            mean_valid_col = max(mean_left_page_num, mean_right_page_num)
            vol_char_df.loc[(vol_char_df["page_num"] == page_num) , "page_num_coord_match"] = vol_char_df[(vol_char_df["page_num"] == page_num)]["pruned_word_bbox"].apply(lambda x : is_coord_match([x[2]], mean_valid_col, mean_valid_col, margin))
        elif mean_left_page_num == -1 and mean_right_page_num == -1:
            vol_char_df.loc[(vol_char_df["page_num"] == page_num) , "page_num_coord_match"] = vol_char_df[(vol_char_df["page_num"] == page_num)]["pruned_word_bbox"].apply(lambda x : False)
        else: 
            vol_char_df.loc[(vol_char_df["page_num"] == page_num) , "page_num_coord_match"] = vol_char_df[(vol_char_df["page_num"] == page_num)]["pruned_word_bbox"].apply(lambda x : is_coord_match([x[2]], mean_left_page_num, mean_right_page_num, margin))


100%|██████████| 28/28 [00:01<00:00, 22.32it/s]


In [99]:
all_vol_data_PN_test = [(vol1_char_df, vol1_index, vol1_doc, "potential_page_num_match_vol1"),
                        (vol2_char_df, vol2_index, vol2_doc, "potential_page_num_match_vol2"),
                        (vol3_char_df, vol3_index, vol3_doc, "potential_page_num_match_vol3")][::-1]

for vol_char_df, vol_index, doc, output_name in all_vol_data_PN_test: 
    #for each volume 
    image_list = []

    for page_num in tqdm(vol_index):
        pix_map = doc.get_page_pixmap(page_num,matrix=mat)
        image = Image.open(io.BytesIO(pix_map.tobytes()))
        draw = ImageDraw.Draw(image)
        

        page_num_coord_db = vol_char_df[(vol_char_df['page_num'] == page_num) & 
                                     (vol_char_df['page_num_coord_match'] == True)
                            ].loc[:,~vol_char_df.columns.isin(["char_num", "char", "char_origin",	"char_bbox"])
                            ].drop_duplicates()

        # infra_db = vol_char_df[(vol_char_df['page_num'] == page_num) 
        #                         & (vol_char_df['potential_infra_match'] == True)
        #                         ].loc[:,~vol_char_df.columns.isin(["char_num", "char", "char_origin",	"char_bbox"])
        #                         ].drop_duplicates()

        # with_infra_symbols = vol_char_df[(vol_char_df['page_num'] == page_num) &
        #                                  (vol_char_df['infra_coord_match'] == True) & 
        #                                  (vol_char_df['word'].apply(has_infra_symbols) == True)
        #                                 ].loc[:,~vol_char_df.columns.isin(["char_num", "char", "char_origin",	"char_bbox"])
        #                                 ].drop_duplicates()

        #genus Coord is orange-pinkish, 5
        for coord in page_num_coord_db['word_bbox'] :
            x0, y0, x1, y1 = [f*TARGET_DPI/ 72 for f in coord]
            draw.rectangle((x0, y0, x1, y1), fill=None, outline=ImageColor.getrgb("#003399"), width=3)

        # for coord in infra_db['word_bbox'] :
        #     x0, y0, x1, y1 = [f*TARGET_DPI/ 72 for f in coord]
        #     draw.rectangle((x0-3, y0-3, x1+3, y1+3), fill=None, outline=ImageColor.getrgb("#FF7F50"), width=5)
            
        # # #epithet is red, 3
        # for coord in with_infra_symbols['word_bbox'] :
        #     x0, y0, x1, y1 = [f*TARGET_DPI/ 72 for f in coord]
        #     draw.rectangle((x0, y0, x1, y1), fill=None, outline=ImageColor.getrgb("#990000"), width=3)

        image_list.append(image)

    #save pages of the volume
    image_list[0].save('../output/local/'+output_name+'.pdf' ,save_all=True, append_images=image_list[1:])

100%|██████████| 23/23 [00:03<00:00,  5.96it/s]


### catching & hardcoding issues

#### checking potential_infra_match that are not with typical symbols


In [100]:
vol1_char_df[(vol1_char_df['potential_infra_match'] == True) & (vol1_char_df['word'].apply(has_infra_symbols) == False)][["index_page_num", "word"]].drop_duplicates()

,index_page_num,word


In [101]:
vol2_char_df[(vol2_char_df['potential_infra_match'] == True) & (vol2_char_df['word'].apply(has_infra_symbols) == False)][["index_page_num", "word"]].drop_duplicates()

,index_page_num,word


In [102]:
vol3_char_df[(vol3_char_df['potential_infra_match'] == True) & (vol3_char_df['word'].apply(has_infra_symbols) == False)][["index_page_num", "word"]].drop_duplicates()

,index_page_num,word
1557763,3,(3
1558282,3,f.
1559900,4,fa
1564627,6,deris
1568637,8,cock
1574726,11,f.
1576530,12,picha
1579429,13,adoxifolium
1582311,15,yar.
1585217,16,f.


In [103]:
remove_infra_index = [1566359, 1570483, 1578491, 1581443, 1582167, 1594835, 1598587]
#temp_df_hard_code_infra = vol3_char_df[(vol3_char_df['potential_infra_match'] == True) & (vol3_char_df['word'].apply(has_infra_symbols) == False)][["index_page_num", "pruned_word"]].drop_duplicates()
#temp_df_hard_code_infra[temp_df_hard_code_infra['pruned_word'].apply(lambda x : len(x) > 3)].index 
#nice ways but won't have everything in them ... so 

In [104]:
def get_index_end(vol_df, start_index):
    len_word = len(vol_df.loc[start_index,'word'])
    print(len_word, start_index + len_word)
    return start_index + len_word - 1

set_epithet_index = [1581443]
remove_infra_index = [1570483, 1566359, 1570883, 1570883, 1578491, 1581443, 1582167, 1594835, 1598587]
for i in remove_infra_index:
    vol3_char_df.loc[i : get_index_end(vol3_char_df, i),'potential_infra_match'] = False

for i in set_epithet_index:
    vol3_char_df.loc[i : get_index_end(vol3_char_df, i),'potential_epithet_match'] = True


10 1570493
10 1566369
8 1570891
8 1570891
3 1578494
6 1581449
9 1582176
7 1594842
3 1598590
6 1581449


In [137]:
#cock keeps needing to be pruned multiple times ....??? not sure why ugh
vol3_char_df[(vol3_char_df['potential_infra_match'] == True) & (vol3_char_df['word'].apply(has_infra_symbols) == False)][["index_page_num", "word"]].drop_duplicates()

,index_page_num,word
1557763,3,(3
1558282,3,f.
1559900,4,fa
1564627,6,deris
1568637,8,cock
1574726,11,f.
1576530,12,picha
1579429,13,adoxifolium
1582311,15,yar.
1585217,16,f.


1596384 -> var with space in between it somehow

In [106]:
ending = get_index_end(vol3_char_df, 1596384)+2
vol3_char_df.loc[1596384 : ending, 'word'] = 'var.'
vol3_char_df.loc[1596384 : ending, 'word_num'] = 0
vol3_char_df.loc[1596384 : ending, 'pruned_word'] = 'var'
vol3_char_df.loc[1596384 : ending, 'potential_infra_match'] = True 
#have to run it again this is problematic now

6 1596390


#### upper case beggining / latin words in epithet coordd

In [107]:
def potential_author_match_epithet_coord(word):
    latin_connectives = r"^\s?et[\s|.]?$|^\s?in[\s|.]?$|^\s?non[\s|.]?$|^\s?&[\s|.]?$|^\s?er[\s|.]?$|^\s?nec[\s|.]?$|^\s?mult[\s|.]?$|^\s?ex[\s|.]?$|^\s?fil[\s|.]?$|^\s?f[\s|.]?$"
    is_latin_connectives = re.search(latin_connectives, word) != None
    is_hybrid = word == "X"
    return is_latin_connectives or (word[0].isupper() and (not is_hybrid))

CHECKED: for vol1 all are okay and are just typos 

In [108]:
vol1_char_df[(vol1_char_df['potential_epithet_match'] == True) & (vol1_char_df['word'].apply(potential_author_match_epithet_coord))][["index_page_num", "word"]].drop_duplicates()

,index_page_num,word
1750223,18,Phoenicia
1750722,18,Syriacus
1752316,19,Jilicaulis


CHECKED Vol2 

Hbanoticus -> typo for libanoticus 

Hppii -> typo for lippii

letting fuzzy matching take care of it later :)

**TODO**: Ma -> is supposed to be chia (not easy for fuzzy matching to fix...)


In [109]:
vol2_char_df[(vol2_char_df['potential_epithet_match'] == True) & (vol2_char_df['word'].apply(potential_author_match_epithet_coord))][["index_page_num", "word"]].drop_duplicates()

,index_page_num,word
1914060,4,Hbanoticus
1921063,8,Hppii
1935567,14,Ma


CHECKED vol3 

In [110]:
vol3_char_df[(vol3_char_df['potential_epithet_match'] == True) & (vol3_char_df['word'].apply(potential_author_match_epithet_coord))][["index_page_num", "word"]].drop_duplicates()

,index_page_num,word
1553087,1,Krascheninnikovii
1563802,6,Wagenitz
1564785,6,Fritsch
1573283,10,Holub
1573581,11,Holub
1575255,11,et
1576974,12,Eichwaldii
1577060,12,Schrank
1580067,14,Kuntze
1581446,14,Chav.


In [111]:
# Eichwaldii, Krascheninnikovii, DOteriifolium (p should be p) -> oki 
# Majoranamaracus -> hybrid situation -> fixed later
not_epithet_index_list = [1565497, 1566524, 1575185, 1575488, 1577207, 1579044, 1582101, 1583001, 1586349, 1586394, 1592464, 1608442, 1609136, 1610970, 1611185]

for i in not_epithet_index_list:
    vol3_char_df.loc[i : get_index_end(vol3_char_df, i),'potential_epithet_match'] = False

5 1565502
6 1566530
11 1575196
3 1575491
12 1577219
5 1579049
11 1582112
3 1583004
6 1586355
15 1586409
13 1592477
6 1608448
3 1609139
13 1610983
3 1611188


In [112]:
vol3_char_df[(vol3_char_df['potential_epithet_match'] == True) & (vol3_char_df['word'].apply(potential_author_match_epithet_coord))][["index_page_num", "word"]].drop_duplicates()

,index_page_num,word
1553087,1,Krascheninnikovii
1563802,6,Wagenitz
1564785,6,Fritsch
1573283,10,Holub
1573581,11,Holub
1575255,11,et
1576974,12,Eichwaldii
1577060,12,Schrank
1580067,14,Kuntze
1581446,14,Chav.


#### [**TODO FIX LATER**] epithet coord word has uppper case in the middle (but not the first letter)

In [113]:
def has_upper_not_first(word):
    return word[1:].lower() != word[1:]

In [114]:
vol1_char_df[(vol1_char_df['potential_epithet_match'] == True) & (vol1_char_df['word'].apply(has_upper_not_first))][["index_page_num", "word"]].drop_duplicates()

,index_page_num,word
1713635,1,peregrina(Hack.)
1713736,1,umbeUulata
1732133,9,elatior'L.
1733993,10,sessUis
1734796,11,pilosaHuds.
1738966,13,phleoides^Vill.)
1744667,15,albaL.
1749322,18,aegUops
1750031,18,glaucaVahl
1758235,22,auCheriana


In [115]:
vol2_char_df[(vol2_char_df['potential_epithet_match'] == True) & (vol2_char_df['word'].apply(has_upper_not_first))][["index_page_num", "word"]].drop_duplicates()

,index_page_num,word
1912660,4,corîdûpUcaÈu^Sretoï.
1916475,6,securidacaiÇL.)
1940242,17,corymbulosum(Planch.)Reichenb.
1951749,22,aqUatilis


In [116]:
vol3_char_df[(vol3_char_df['potential_epithet_match'] == True) & (vol3_char_df['word'].apply(has_upper_not_first))][["index_page_num", "word"]].drop_duplicates()

,index_page_num,word
1566755,7,gaiUardotii
1577865,13,albu^L.
1583549,15,sieberiC.
1584228,16,Schiman-Czeika
1595344,21,desertiTUéh.
1601261,24,DOteriifolium
1605792,26,agMmoniifolium
1610246,28,'Abd-el-'asissi


#### hardcoding Genus Platanthera

In [117]:
vol1_char_df[(vol1_char_df['potential_genus_match'] == True) & (vol1_char_df['word'] == 'Platan')][['word_num', 'char_num']]

,word_num,char_num
1746680,0,0
1746681,0,1
1746682,0,2
1746683,0,3
1746684,0,4
1746685,0,5


In [118]:
vol1_char_df.shape[0]

1761804

In [121]:
vol1_char_df[(vol1_char_df['potential_genus_match'] == True) & (vol1_char_df['word'] == 'Platan')][['page_num', 'block_num','line_num']].iloc[0]

page_num     631
block_num     87
line_num       0
Name: 1746680, dtype: int64

In [122]:
vol1_char_df[(vol1_char_df['page_num'] == 631) & (vol1_char_df['block_num'] == 87) & (vol1_char_df['line_num'] == 0)][['page_num', 'block_num','line_num', 'word', 'word_num', 'char', 'span_num']]

,page_num,block_num,line_num,word,word_num,char,span_num
1746680,631,87,0,Platan,0,P,0
1746681,631,87,0,Platan,0,l,0
1746682,631,87,0,Platan,0,a,0
1746683,631,87,0,Platan,0,t,0
1746684,631,87,0,Platan,0,a,0
1746685,631,87,0,Platan,0,n,0
1746686,631,87,0,thera,1,t,0
1746687,631,87,0,thera,1,h,0
1746688,631,87,0,thera,1,e,0
1746689,631,87,0,thera,1,r,0


In [123]:
vol1_char_df.loc[(vol1_char_df['page_num'] == 631) & (vol1_char_df['block_num'] == 87) & (vol1_char_df['line_num'] == 0),'potential_genus_match'] = True
vol1_char_df.loc[(vol1_char_df['page_num'] == 631) & (vol1_char_df['block_num'] == 87) & (vol1_char_df['line_num'] == 0),'potential_epithet_match'] = False
vol1_char_df.loc[(vol1_char_df['page_num'] == 631) & (vol1_char_df['block_num'] == 87) & (vol1_char_df['line_num'] == 0),'word_num'] = 0
vol1_char_df.loc[(vol1_char_df['page_num'] == 631) & (vol1_char_df['block_num'] == 87) & (vol1_char_df['line_num'] == 0),'word'] = 'Platanthera'
vol1_char_df.loc[(vol1_char_df['page_num'] == 631) & (vol1_char_df['block_num'] == 87) & (vol1_char_df['line_num'] == 0),'pruned_word'] = 'Platanthera'

In [124]:
vol1_char_df.loc[1746680, 'word']

'Platanthera'

In [125]:
# keep_cols = vol1_char_df.columns.difference(["char_num", "char", "char_origin", "char_bbox"])
# first_entry = vol1_char_df.loc[(vol1_char_df['page_num'] == 631) & (vol1_char_df['block_num'] == 87) & (vol1_char_df['line_num'] == 0)].iloc[0]
# for col_name in keep_cols: 
#     print(vol1_char_df.loc[(vol1_char_df['page_num'] == 631) & (vol1_char_df['block_num'] == 87) & (vol1_char_df['line_num'] == 0), col_name], first_entry[col_name])
#     vol1_char_df.loc[(vol1_char_df['page_num'] == 631) & (vol1_char_df['block_num'] == 87) & (vol1_char_df['line_num'] == 0), col_name] = first_entry[col_name]
#     #print(first_entry[col_name])

In [126]:
vol1_char_df[(vol1_char_df['page_num'] == 631) & (vol1_char_df['block_num'] == 87) & (vol1_char_df['line_num'] == 0)][['page_num', 'block_num','line_num', 'word', 'word_num', 'char', 'char_num']]

,page_num,block_num,line_num,word,word_num,char,char_num
1746680,631,87,0,Platanthera,0,P,0
1746681,631,87,0,Platanthera,0,l,1
1746682,631,87,0,Platanthera,0,a,2
1746683,631,87,0,Platanthera,0,t,3
1746684,631,87,0,Platanthera,0,a,4
1746685,631,87,0,Platanthera,0,n,5
1746686,631,87,0,Platanthera,0,t,0
1746687,631,87,0,Platanthera,0,h,1
1746688,631,87,0,Platanthera,0,e,2
1746689,631,87,0,Platanthera,0,r,3


In [127]:
# make same 
# iterate -> char_num = list(range(df.shape[0]))
# get max 
# get min 

In [128]:
def copy_data(df, src_index, dst_indecies, col_name, inplace = True):
    """ used for modifying columns of datatype that can be dirrectly assigned to multiple rows
        INPUT: 
              - df: target dataframe to modify 
              - 
        OUTPUT: returns df 
    """
    if inplace == False:
        df = df.copy()
    
    src_value = df.loc[src_index, col_name]
    if isinstance(src_value, tuple):
        df.loc[dst_indecies, col_name] = [src_value] * len(dst_indecies)
    else: 
        df.loc[dst_indecies, col_name] = src_value

    return df 

In [129]:
def copy_tuple_data(df, src_index, dst_indecies, col_name, inplace = True):
    """ Used for modifying columns with lists/tuples
        INPUT: 
              - df: target dataframe to modify 
              - 
        OUTPUT: returns df 
    """
    if inplace == False:
        df = df.copy()
    
    src_value = df.loc[src_index, col_name]
    df.loc[dst_indecies, col_name] = [src_value] * len(dst_indecies)

    return df 

In [130]:
def reiterate(df, src_index, dst_indecies, col_name, inplace = True):
    """ Used for modifying columns with lists/tuples
        INPUT: 
              - df: target dataframe to modify 
              - 
        OUTPUT: returns df 
    """
    if inplace == False:
        df = df.copy()
    
    df.loc[dst_indecies, col_name] = list(range(len(dst_indecies)))

    return df 

In [131]:
def transform_values(df, src_index, dst_indecies, col_name, transformations, inplace = True):
    """ Used for modifying columns with lists/tuples
        INPUT: 
              - df: target dataframe to modify 
              - 
        OUTPUT: returns df 
    """
    if inplace == False:
        df = df.copy()

    # if src_value is of type list or tuple or something like that, make sure transformations are available for each index or the size is 1. 
    # the apply transformations to each index
    

    df.loc[dst_indecies, col_name]

    src_value = df.loc[src_index, col_name]
    
    df.loc[dst_indecies, col_name] = src_value

    return df 

In [132]:
def merge_bbox(df, indecies, col_name, inplace = True):
    """ Used for modifying columns with lists/tuples
        INPUT: 
              - df: target dataframe to modify 
              - 
        OUTPUT: returns df 
    """
    if inplace == False:
        df = df.copy()

    # if src_value is of type list or tuple or something like that, make sure transformations are available for each index or the size is 1. 
    # the apply transformations to each index
    

    x0 = df.loc[indecies, col_name].apply(lambda x: x[0]).min()
    y0 = df.loc[indecies, col_name].apply(lambda x: x[1]).min()
    x1 = df.loc[indecies, col_name].apply(lambda x: x[2]).max()
    y1 = df.loc[indecies, col_name].apply(lambda x: x[3]).max()
    

    new_bbox = (x0, y0, x1, y1)
    df.loc[indecies, col_name] = [new_bbox] * len(indecies)

    return df 

In [134]:
def merge_str(df, src_index, dst_indecies, col_name, inplace = True):
    
    return 


In [135]:
vol1_char_df.loc[100, 'word']

'scientifiques'

In [136]:
# need to fix anything vol_index_df.columns.difference(["char_num", "char", "char_origin", "char_bbox"]

#### Potential genus mismatch

potential genus match but name is not alphabetic or is of length < 3

In [538]:
def flag_genus_name(word):
    word_no_space = word.replace(" ", "")
    return ((not word_no_space.isalpha()) or (len(word_no_space) < 3))

In [539]:
vol1_char_df[(vol1_char_df['potential_genus_match'] == True) & (vol1_char_df['word'].apply(flag_genus_name))][["index_page_num", "word"]].drop_duplicates()
#skipping over all these 

,index_page_num,word
1728448,8,c
1736085,11,f
1751898,19,j.


In [540]:
not_genus_index_list_vol1 = [1730943, 1738656, 1754704]
for i in not_genus_index_list_vol1:
    vol1_char_df.loc[i : get_index_end(vol1_char_df, i),'potential_genus_match'] = False

7 1730950
9 1738665
4 1754708


In [541]:
vol1_char_df[(vol1_char_df['potential_genus_match'] == True) & (vol1_char_df['word'].apply(flag_genus_name))][["index_page_num", "word"]].drop_duplicates()

,index_page_num,word
1728448,8,c
1736085,11,f
1751898,19,j.


In [542]:
vol2_char_df[(vol2_char_df['potential_genus_match'] == True) & (vol2_char_df['word'].apply(flag_genus_name))][["index_page_num", "word"]].drop_duplicates()

,index_page_num,word
1919791,7,•Ceratophyllum
1920312,7,Chelidonium^
1937882,16,Jussiaea-
1938004,16,VV.1l.*


In [543]:
not_genus_index_list_vol2 = [1939606]
for i in not_genus_index_list_vol2:
    vol2_char_df.loc[i : get_index_end(vol2_char_df, i),'potential_genus_match'] = False

7 1939613


In [544]:
vol2_char_df[(vol2_char_df['potential_genus_match'] == True) & (vol2_char_df['word'].apply(flag_genus_name))][["index_page_num", "word"]].drop_duplicates()

,index_page_num,word
1919791,7,•Ceratophyllum
1920312,7,Chelidonium^
1937882,16,Jussiaea-
1938004,16,VV.1l.*


In [545]:
vol3_char_df[(vol3_char_df['potential_genus_match'] == True) & (vol3_char_df['word'].apply(flag_genus_name))][["index_page_num", "word"]].drop_duplicates()
#all okay and hybrid will get fixed later :)

,index_page_num,word
1558145,3,BallotaL.
1568136,8,CordiaL.
1582571,15,x
1599304,23,SolidagoL.


In [546]:
vol1_char_df[(vol1_char_df['potential_genus_match'] == True) & (vol1_char_df['word'].apply(lambda x : x[0].isupper() == False))]['word']

1728448          c
1736085          f
1751898         j.
1751899         j.
1756566    apetala
1756567    apetala
1756568    apetala
1756569    apetala
1756570    apetala
1756571    apetala
1756572    apetala
Name: word, dtype: object

In [547]:
index_apetala_correcting = vol1_char_df[(vol1_char_df['potential_genus_match'] == True) & (vol1_char_df['word'].apply(lambda x : x[0].isupper() == False))].index
vol1_char_df.loc[index_apetala_correcting,'potential_genus_match'] = False
vol1_char_df.loc[index_apetala_correcting,'potential_epithet_match'] = True

In [548]:
vol1_char_df[(vol1_char_df['potential_genus_match'] == True) & (vol1_char_df['word'].apply(lambda x : x[0].isupper() == False))]['word']

Series([], Name: word, dtype: object)

In [549]:
#fuzzy matching will fix
vol2_char_df[(vol2_char_df['potential_genus_match'] == True) & (vol2_char_df['word'].apply(lambda x : x[0].isupper() == False))]['word']

1919791    •Ceratophyllum
1919792    •Ceratophyllum
1919793    •Ceratophyllum
1919794    •Ceratophyllum
1919795    •Ceratophyllum
1919796    •Ceratophyllum
1919797    •Ceratophyllum
1919798    •Ceratophyllum
1919799    •Ceratophyllum
1919800    •Ceratophyllum
1919801    •Ceratophyllum
1919802    •Ceratophyllum
1919803    •Ceratophyllum
1919804    •Ceratophyllum
Name: word, dtype: object

In [550]:
#fixed later when hybrids identified 
vol3_char_df[(vol3_char_df['potential_genus_match'] == True) & (vol3_char_df['word'].apply(lambda x : x[0].isupper() == False))]['word']

1582571    x
Name: word, dtype: object

### pruning char_df and getting index_df

In [551]:
[c for c in vol1_char_df.columns if c.startswith('potential')]

['potential_genus_match', 'potential_epithet_match', 'potential_infra_match']

In [552]:
#making sure page_num is in index
#making sure the genus level word is not all uppercase (a family name)
#making sure the pruned_word is not numeric (removing page_number as it's not in order usually) and removing page_num_coord_match

all_vol_data = [(vol1_char_df, vol1_index),
                (vol2_char_df, vol2_index),
                (vol3_char_df, vol3_index)]

result = [] 
ignore_word_list = ["NOUVELLE", "Flore", "FLORE", "INDEX", ""]
for vol_char_df, vol_index in all_vol_data:
    curr_result_df = vol_char_df[(vol_char_df['page_num'].isin(vol_index)) &
                                (~((vol_char_df["word"].str.isupper()) & (vol_char_df["word"].apply(lambda x : len(x) > 2)) & (vol_char_df['genus_coord_match'] == True))) & 
                                (~(vol_char_df["pruned_word"].isin(ignore_word_list))) &
                                (~(vol_char_df["pruned_word"].str.isnumeric() & (vol_char_df["word"] != "(3"))) & 
                                (~(vol_char_df["page_num_coord_match"] == True)) & 
                                (~((vol_char_df.groupby(['page_num', 'block_num', 'line_num'])['char_num'].transform('max') == 0) & (vol_char_df['word'].str.isupper())))
                                ].copy()
    result.append(curr_result_df)

vol1_index_df, vol2_index_df, vol3_index_df = result[0], result[1], result[2]

In [553]:
all_vol_data_PN_test = [(vol1_index_df, vol1_index, vol1_doc, "valid_words_vol1"),
                        (vol2_index_df, vol2_index, vol2_doc, "valid_words_vol2"),
                        (vol3_index_df, vol3_index, vol3_doc, "valid_words_vol3")]

for vol_char_df, vol_index, doc, output_name in all_vol_data_PN_test: 
    #for each volume 
    image_list = []

    for page_num in tqdm(vol_index):
        pix_map = doc.get_page_pixmap(page_num,matrix=mat)
        image = Image.open(io.BytesIO(pix_map.tobytes()))
        draw = ImageDraw.Draw(image)
        temp_coords = vol_char_df[vol_char_df['page_num'] == page_num]['word_bbox'].drop_duplicates()
        for coord in temp_coords:
            x0, y0, x1, y1 = [f*TARGET_DPI/ 72 for f in coord]
            draw.rectangle((x0, y0, x1, y1), fill=None, outline=ImageColor.getrgb("#003399"), width=3)

        image_list.append(image)

    #save pages of the volume
    image_list[0].save('../output/local/'+output_name+'.pdf' ,save_all=True, append_images=image_list[1:])

100%|██████████| 28/28 [00:04<00:00,  6.47it/s]


In [554]:
#only keeping word level
vol_index_df_list = [vol1_index_df, vol2_index_df, vol3_index_df]
result_df = []
for vol_index_df in vol_index_df_list:
    keep_cols = vol_index_df.columns.difference(["char_num", "char", "char_origin", "char_bbox"], sort=False).tolist()

    vol_index_df = vol_index_df.copy().loc[:,keep_cols].drop_duplicates().reset_index()
    vol_index_df.rename(columns={"index": "char_index"}, inplace = True)
    result_df.append(vol_index_df)

vol1_index_df, vol2_index_df, vol3_index_df = result_df[0], result_df[1], result_df[2]


In [555]:
def has_hybrid_symbols(word):
    infra_symbols = r"^X[\s|.|\b]?$|^x[\s|.|\b]?$|^×[\s|.|\b]?$"
    return re.search(infra_symbols, word) != None

In [556]:
result_df_hybrids = []
for vol_index_df in [vol1_index_df, vol2_index_df, vol3_index_df]:
    vol_index_df['is_hybrid'] = np.NaN
    vol_index_df.loc[(vol_index_df['potential_infra_match'] == True) | (vol_index_df['potential_epithet_match'] == True) | (vol_index_df['potential_genus_match'] == True), 'is_hybrid'] = (vol_index_df['word'].apply(has_hybrid_symbols) == True) & ((vol_index_df['potential_infra_match'] == True) | (vol_index_df['potential_epithet_match'] == True) | (vol_index_df['potential_genus_match'] == True))
    
    hybrid_genera_indecies = vol_index_df[(vol_index_df['potential_genus_match'] == True) & (vol_index_df['word'].apply(has_hybrid_symbols) == True)].index + 1
    hybrid_epithet_indecies = vol_index_df[(vol_index_df['potential_epithet_match'] == True) & (vol_index_df['word'].apply(has_hybrid_symbols) == True)].index + 1
    
    vol_index_df.loc[hybrid_epithet_indecies, 'is_hybrid'] = True
    vol_index_df.loc[hybrid_epithet_indecies, 'potential_epithet_match'] = True 

    vol_index_df.loc[hybrid_genera_indecies, 'is_hybrid'] = True
    vol_index_df.loc[hybrid_genera_indecies, 'potential_genus_match'] = True

    drop_list = list(hybrid_epithet_indecies - 1) + list(hybrid_genera_indecies -1)
    
    vol_index_df = vol_index_df[~vol_index_df.index.isin(drop_list)].copy()
    #vol_index_df['is_hybrid'].ffill(inplace=True) fowrward fill after checking hybrid for the infra species types too

    result_df_hybrids.append(vol_index_df)

vol1_index_df, vol2_index_df, vol3_index_df = result_df_hybrids[0], result_df_hybrids[1], result_df_hybrids[2]

In [557]:
#df['closest_epithet_v2'] = np.nan
def extract_potential_genus_names(row):
    if row['potential_genus_match'] == True:
        return row['word'] + "_" + str(row['page_num']) + "_" + str(row['block_num']) + "_" + str(row['line_num'])
    else:
        return np.nan
        
for vol_index_df in [vol1_index_df, vol2_index_df, vol3_index_df]:
    vol_index_df['closest_genus'] = vol_index_df.apply(extract_potential_genus_names, axis = 1)
    vol_index_df['closest_genus'].ffill(inplace=True)

In [558]:
#df['closest_epithet_v2'] = np.nan
def extract_potential_epithet_names(row):
    if row['potential_epithet_match'] == True:
        return row['word'] + "_" + str(row['page_num']) + "_" + str(row['block_num']) + "_" + str(row['line_num'])
    else:
        return np.nan

for vol_index_df in [vol1_index_df, vol2_index_df, vol3_index_df]:
    vol_index_df['closest_epithet'] = vol_index_df.apply(extract_potential_epithet_names, axis = 1)
    vol_index_df.loc[vol_index_df['potential_genus_match'] == True, 'closest_epithet'] = -1
    vol_index_df['closest_epithet'].ffill(inplace=True)

In [559]:
def extract_potential_infra_type(row):
    if row['potential_infra_match'] == True:
        return row['word'] + "_" + str(row['page_num']) + "_" + str(row['block_num']) + "_" + str(row['line_num'])
    else:
        return np.nan

for vol_index_df in [vol1_index_df, vol2_index_df, vol3_index_df]:
    vol_index_df.loc[(vol_index_df['potential_epithet_match'] == True) | (vol_index_df['potential_genus_match'] == True), 'closest_infra_type'] = -1
    vol_index_df['closest_infra_type'] = vol_index_df.apply(extract_potential_infra_type, axis = 1)
    vol_index_df.loc[(vol_index_df['potential_infra_match'] == False) & ((vol_index_df['potential_epithet_match'] == True) | (vol_index_df['potential_genus_match'] == True)), 'closest_infra_type'] = -1
    vol_index_df['closest_infra_type'].ffill(inplace=True)

In [560]:
for vol_index_df in [vol1_index_df, vol2_index_df, vol3_index_df]:
    infra_name_match_indecies = vol_index_df[vol_index_df['potential_infra_match'] == True].index + 1
    vol_index_df['closest_infra_name'] = np.NaN
    vol_index_df.loc[infra_name_match_indecies, 'closest_infra_name'] = vol_index_df.apply(lambda row : row['word'] + "_" + str(row['page_num']) + "_" + str(row['block_num']) + "_" + str(row['line_num']) , axis = 1)
    vol_index_df['potential_infra_name_match'] = vol_index_df.index.isin(infra_name_match_indecies)
    vol_index_df.loc[(vol_index_df['potential_infra_match'] == True) | (vol_index_df['potential_epithet_match'] == True) | (vol_index_df['potential_genus_match'] == True), 'closest_infra_name'] = -1
    vol_index_df['closest_infra_name'].ffill(inplace=True)

In [561]:
for vol_index_df in [vol1_index_df, vol2_index_df, vol3_index_df]:
    cond = ((vol_index_df['closest_infra_name'] != '') | (vol_index_df['closest_infra_name'] != -1) | (~vol_index_df['closest_infra_name'].isna())) & \
           (vol_index_df['word'].apply(has_hybrid_symbols) == True)
    
    vol_index_df.loc[cond, 'is_hybrid'] = True
    vol_index_df['is_hybrid'].ffill(inplace=True)

In [562]:
for vol_index_df in [vol1_index_df, vol2_index_df, vol3_index_df]:
    vol_index_df['potential_author_match'] = (vol_index_df['potential_genus_match'] == False) & \
                                             (vol_index_df['potential_epithet_match'] == False) & \
                                             (vol_index_df['potential_infra_match'] == False) & \
                                             (vol_index_df['potential_infra_name_match'] == False)

In [563]:
for vol_index_df in [vol1_index_df, vol2_index_df, vol3_index_df]:
    vol_index_df.replace(-1, np.NaN, inplace = True)
    vol_index_df.replace(np.NaN, "",inplace = True)

In [564]:
#author grouping 
# 
author_grouping = ['closest_genus', 'closest_epithet', 'closest_infra_name']
merge_on = ['closest_genus', 'closest_epithet', 'closest_infra_name']
def concatenate(group):
    return group.loc[group['potential_author_match'] == True, 'word'].str.cat(sep=' ')

result_df_authors = [] 
for vol_index_df in [vol1_index_df, vol2_index_df, vol3_index_df]: 
    #author_grouping = ['closest_genus', 'closest_epithet']
    #merge_on = ['closest_genus', 'closest_epithet']
    groups = vol_index_df.groupby(author_grouping)
    concatenated = groups.apply(concatenate).reset_index()

    # add the concatenated values to the original dataframe
    result = vol_index_df.merge(concatenated[merge_on + [0]], on=merge_on, how='left').rename(columns={0: 'authors'})
    result_df_authors.append(result)
    
vol1_index_df, vol2_index_df, vol3_index_df = result_df_authors[0], result_df_authors[1], result_df_authors[2]


In [565]:
# for vol_index_df in [vol1_index_df, vol2_index_df, vol3_index_df]:
#     #vol_index_df.replace("", np.NaN,inplace = True)
#     vol_index_df.replace(np.NaN, "",inplace = True)

In [566]:
all_vol_data_cat_test = [(vol1_index_df, vol1_index, vol1_doc, "catagorized_vol1"),
                         (vol2_index_df, vol2_index, vol2_doc, "catagorized_vol2"),
                         (vol3_index_df, vol3_index, vol3_doc, "catagorized_vol3")]

for vol_char_df, vol_index, doc, output_name in all_vol_data_cat_test: 
    #for each volume 
    image_list = []

    for page_num in tqdm(vol_index):
        pix_map = doc.get_page_pixmap(page_num,matrix=mat)
        image = Image.open(io.BytesIO(pix_map.tobytes()))
        draw = ImageDraw.Draw(image)

        for col_num in [0, 1]:
            temp_df = vol_char_df[(vol_char_df['page_num'] == page_num) & (vol_char_df['col_num'] == col_num)]
            #genus Coord is orange-pinkish, 5
            for name, group in temp_df.groupby(['closest_genus'])['word_bbox']:
                x0 = (group.apply(lambda x : x[0]).min())*TARGET_DPI/ 72
                y0 = (group.apply(lambda x : x[1]).min())*TARGET_DPI/ 72
                x1 = (group.apply(lambda x : x[2]).max())*TARGET_DPI/ 72
                y1 = (group.apply(lambda x : x[3]).max())*TARGET_DPI/ 72
                draw.rectangle((x0, y0, x1, y1), fill=None, outline=ImageColor.getrgb("#6939a3"), width=3)

            for name, group in temp_df.groupby(['closest_epithet'])['word_bbox']:
                if name != '':
                    x0 = (group.apply(lambda x : x[0]).min())*TARGET_DPI/ 72
                    y0 = (group.apply(lambda x : x[1]).min())*TARGET_DPI/ 72
                    x1 = (group.apply(lambda x : x[2]).max())*TARGET_DPI/ 72
                    y1 = (group.apply(lambda x : x[3]).max())*TARGET_DPI/ 72
                    draw.rectangle((x0, y0, x1, y1), fill=None, outline=ImageColor.getrgb("#003399"), width=3)

            for name, group in temp_df.groupby(['closest_infra_name'])['word_bbox']:
                if name != '':
                    x0 = (group.apply(lambda x : x[0]).min())*TARGET_DPI/ 72
                    y0 = (group.apply(lambda x : x[1]).min())*TARGET_DPI/ 72
                    x1 = (group.apply(lambda x : x[2]).max())*TARGET_DPI/ 72
                    y1 = (group.apply(lambda x : x[3]).max())*TARGET_DPI/ 72
                    draw.rectangle((x0, y0, x1, y1), fill=None, outline=ImageColor.getrgb("#8c690b"), width=3)

            temp_df_author_only = temp_df[temp_df['potential_author_match'] == True]
            for name, group in temp_df_author_only.groupby(['closest_genus', 'closest_epithet', 'closest_infra_name'])['word_bbox']:
                x0 = (group.apply(lambda x : x[0]).min())*TARGET_DPI/ 72
                y0 = (group.apply(lambda x : x[1]).min())*TARGET_DPI/ 72
                x1 = (group.apply(lambda x : x[2]).max())*TARGET_DPI/ 72
                y1 = (group.apply(lambda x : x[3]).max())*TARGET_DPI/ 72

                draw.rectangle((x0, y0, x1, y1), fill=None, outline=ImageColor.getrgb("#9e9e9e"), width=3)


        image_list.append(image)

    #save pages of the volume
    image_list[0].save('../output/local/'+output_name+'.pdf' ,save_all=True, append_images=image_list[1:])

100%|██████████| 28/28 [00:05<00:00,  4.86it/s]


In [567]:
def fix_words(word):
    head, sep, tail = word.partition('_')
    return head 

for vol_index_df in [vol1_index_df, vol2_index_df, vol3_index_df]:
    vol_index_df['closest_genus'] = vol_index_df['closest_genus'].apply(fix_words)
    vol_index_df['closest_epithet'] = vol_index_df['closest_epithet'].apply(fix_words)
    vol_index_df['closest_infra_type'] = vol_index_df['closest_infra_type'].apply(fix_words)
    vol_index_df['closest_infra_name'] = vol_index_df['closest_infra_name'].apply(fix_words)

In [568]:
result_prune_authors_list = []
for vol_index_df in [vol1_index_df, vol2_index_df, vol3_index_df]:
    result_prune_authors = vol_index_df[(vol_index_df['potential_genus_match'] == True) |
                                        (vol_index_df['potential_epithet_match'] == True) |
                                        (vol_index_df['potential_infra_name_match'] == True)].copy()
    result_prune_authors_list.append(result_prune_authors)

prune_authors_vol1, prune_authors_vol2, prune_authors_vol3 =  result_prune_authors_list[0], result_prune_authors_list[1], result_prune_authors_list[2]

In [569]:
#quick fix for vol1 epithet on the same line as author situation
cond = (prune_authors_vol1['closest_genus'] != '') & (prune_authors_vol1['closest_epithet'] == '') & (prune_authors_vol1['authors'] != '')
prune_authors_vol1.loc[cond, 'closest_epithet'] = prune_authors_vol1.loc[cond,'authors'].str.split().apply(lambda s: s[0])
prune_authors_vol1.loc[cond, 'authors'] = prune_authors_vol1.loc[cond,'authors'].str.split().apply(lambda s: " ".join(s[1:]))

In [570]:
def get_taxon_rank_specific(row):
    has_genus = (pd.isnull(row['closest_genus']) == False) & (row['closest_genus'] != "") & (row['closest_genus'] != -1)
    has_epithet = (pd.isnull(row['closest_epithet']) == False) & (row['closest_epithet'] != "") & (row['closest_epithet'] != -1)
    
    has_infra = (pd.isnull(row['closest_infra_name']) == False) & (row['closest_infra_name'] != "") & (row['closest_infra_name'] != -1)
    has_infra_type = (pd.isnull(row['closest_infra_type']) == False) & (row['closest_infra_type'] != "") & (row['closest_infra_type'] != -1)
    infra_type = row['closest_infra_type']
    is_infra_hybrid = has_hybrid_symbols(row['closest_infra_type']) == True
    if is_infra_hybrid:
        infra_type = "hybrid"
    
    is_hybrid  = row['is_hybrid'] == True
    prefix  = ""
    if is_hybrid:
        prefix = "hybrid "

    if has_infra or has_infra_type:
        return f"infra ({infra_type})"
    if has_epithet:
        return prefix + "epithet"
    if has_genus:
        return prefix + "genus"

def get_taxon_rank_general(row):
    has_genus = (pd.isnull(row['closest_genus']) == False) & (row['closest_genus'] != "") & (row['closest_genus'] != -1)
    has_epithet = (pd.isnull(row['closest_epithet']) == False) & (row['closest_epithet'] != "") & (row['closest_epithet'] != -1)
    has_infra = (pd.isnull(row['closest_infra_name']) == False) & (row['closest_infra_name'] != "") & (row['closest_infra_name'] != -1)
    has_infra_type = (pd.isnull(row['closest_infra_type']) == False) & (row['closest_infra_type'] != "") & (row['closest_infra_type'] != -1)
    
    if has_infra or has_infra_type:
        return "infra"
    if has_epithet:
        return "epithet"
    if has_genus:
        return "genus"

In [571]:
prune_authors_vol1['closest_genus'] = prune_authors_vol1['closest_genus'].str.replace("œ", "oe" )
prune_authors_vol2['closest_genus'] = prune_authors_vol2['closest_genus'].str.replace("œ", "oe" )
prune_authors_vol3['closest_genus'] = prune_authors_vol3['closest_genus'].str.replace("œ", "oe" )

prune_authors_vol1['closest_epithet'] = prune_authors_vol1['closest_epithet'].str.replace("œ", "oe" )
prune_authors_vol2['closest_epithet'] = prune_authors_vol2['closest_epithet'].str.replace("œ", "oe" )
prune_authors_vol3['closest_epithet'] = prune_authors_vol3['closest_epithet'].str.replace("œ", "oe" )

prune_authors_vol1['closest_infra_name'] = prune_authors_vol1['closest_infra_name'].str.replace("œ", "oe" )
prune_authors_vol2['closest_infra_name'] = prune_authors_vol2['closest_infra_name'].str.replace("œ", "oe" )
prune_authors_vol3['closest_infra_name'] = prune_authors_vol3['closest_infra_name'].str.replace("œ", "oe" )

In [572]:
for vol_index_df in [prune_authors_vol1, prune_authors_vol2, prune_authors_vol3]:
    vol_index_df['taxon_rank'] = vol_index_df.apply(get_taxon_rank_general, axis = 1)
    vol_index_df['taxon_rank_detailed'] = vol_index_df.apply(get_taxon_rank_specific, axis = 1)

In [573]:
simplified_vol1 = prune_authors_vol1[['closest_genus',
                                      'closest_epithet',
                                      'closest_infra_name',
                                      'authors',
                                      'taxon_rank',
                                      'taxon_rank_detailed']]
simplified_vol1.to_csv('../output/local/index_output/vol1_index_output.csv')

simplified_vol2 = prune_authors_vol2[['closest_genus',
                                      'closest_epithet',
                                      'closest_infra_name',
                                      'authors',
                                      'taxon_rank',
                                      'taxon_rank_detailed']]
simplified_vol2.to_csv('../output/local/index_output/vol2_index_output.csv')
                                
simplified_vol3 = prune_authors_vol3[['closest_genus',
                                      'closest_epithet',
                                      'closest_infra_name',
                                      'authors',
                                      'taxon_rank',
                                      'taxon_rank_detailed']]
simplified_vol3.to_csv('../output/local/index_output/vol3_index_output.csv')

In [574]:
non_italics_simplified_vol1 = prune_authors_vol1.loc[(prune_authors_vol1['span_flags'] != 6),
                                                     ['closest_genus',
                                                      'closest_epithet',
                                                      'closest_infra_name',
                                                      'authors',
                                                      'taxon_rank',
                                                      'taxon_rank_detailed']]
non_italics_simplified_vol1.to_csv('../output/local/index_output/vol1_nonitalics.csv')

non_italics_simplified_vol2 = prune_authors_vol2.loc[(prune_authors_vol2['span_flags'] != 6),
                                                     ['closest_genus',
                                                      'closest_epithet',
                                                      'closest_infra_name',
                                                      'authors',
                                                      'taxon_rank',
                                                      'taxon_rank_detailed']]
non_italics_simplified_vol2.to_csv('../output/local/index_output/vol2_nonitalics.csv')

non_italics_simplified_vol3 = prune_authors_vol3.loc[(prune_authors_vol3['span_flags'] != 6),
                                                     ['closest_genus',
                                                      'closest_epithet',
                                                      'closest_infra_name',
                                                      'authors',
                                                      'taxon_rank',
                                                      'taxon_rank_detailed']]
non_italics_simplified_vol3.to_csv('../output/local/index_output/vol3_nonitalics.csv')